In [2]:
import numpy as np
import pandas as pd     
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns    
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVR
from sklearn.metrics import r2_score,mean_squared_error

In [3]:
df = pd.read_csv("latimes-county-totals.csv")

In [4]:
df.head()

,date,county,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
0,2020-01-26,Alameda,1,0,0,NaN,NaN
1,2020-01-31,Alameda,1,0,0,0.0,0.0
2,2020-02-02,Alameda,1,0,0,0.0,0.0
3,2020-02-20,Alameda,1,0,0,0.0,0.0
4,2020-02-21,Alameda,1,0,0,0.0,0.0


In [5]:
# df.describe()

In [6]:
# df.info()

In [7]:
df_la = df[df['county'] == 'Los Angeles']

In [8]:
print(df_la)

           date       county  fips  confirmed_cases  deaths  \
795  2020-01-26  Los Angeles    37                1       0   
796  2020-01-27  Los Angeles    37                1       0   
797  2020-01-28  Los Angeles    37                1       0   
798  2020-01-29  Los Angeles    37                1       0   
799  2020-01-30  Los Angeles    37                1       0   
..          ...          ...   ...              ...     ...   
886  2020-04-26  Los Angeles    37            19567     916   
887  2020-04-27  Los Angeles    37            20460     948   
888  2020-04-28  Los Angeles    37            21017    1004   
889  2020-04-29  Los Angeles    37            22522    1065   
890  2020-04-30  Los Angeles    37            23233    1119   

     new_confirmed_cases  new_deaths  
795                  NaN         NaN  
796                  0.0         0.0  
797                  0.0         0.0  
798                  0.0         0.0  
799                  0.0         0.0  
..       

In [9]:
days = list(range(1,len(df_la)+1))
len(days)

96

In [10]:
df_la['culm_day'] = days
#print(df_la)

# df_la = df_la.loc[df_la.culm_day == df_la.date].copy()


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_la

,date,county,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths,culm_day
795,2020-01-26,Los Angeles,37,1,0,NaN,NaN,1
796,2020-01-27,Los Angeles,37,1,0,0.0,0.0,2
797,2020-01-28,Los Angeles,37,1,0,0.0,0.0,3
798,2020-01-29,Los Angeles,37,1,0,0.0,0.0,4
799,2020-01-30,Los Angeles,37,1,0,0.0,0.0,5
...,...,...,...,...,...,...,...,...
886,2020-04-26,Los Angeles,37,19567,916,408.0,20.0,92
887,2020-04-27,Los Angeles,37,20460,948,893.0,32.0,93
888,2020-04-28,Los Angeles,37,21017,1004,557.0,56.0,94
889,2020-04-29,Los Angeles,37,22522,1065,1505.0,61.0,95


In [12]:
df_la.columns

Index(['date', 'county', 'fips', 'confirmed_cases', 'deaths',
       'new_confirmed_cases', 'new_deaths', 'culm_day'],
      dtype='object')

In [13]:
df_la = df_la.drop(labels=['date','county','fips'], axis=1)
# print (df_la.info())
# df_la.describe()

In [14]:
df_la

,confirmed_cases,deaths,new_confirmed_cases,new_deaths,culm_day
795,1,0,NaN,NaN,1
796,1,0,0.0,0.0,2
797,1,0,0.0,0.0,3
798,1,0,0.0,0.0,4
799,1,0,0.0,0.0,5
...,...,...,...,...,...
886,19567,916,408.0,20.0,92
887,20460,948,893.0,32.0,93
888,21017,1004,557.0,56.0,94
889,22522,1065,1505.0,61.0,95


In [15]:
# df_la.dropna(inplace=True)
df_la = df_la.fillna(0)

In [16]:
df_la

,confirmed_cases,deaths,new_confirmed_cases,new_deaths,culm_day
795,1,0,0.0,0.0,1
796,1,0,0.0,0.0,2
797,1,0,0.0,0.0,3
798,1,0,0.0,0.0,4
799,1,0,0.0,0.0,5
...,...,...,...,...,...
886,19567,916,408.0,20.0,92
887,20460,948,893.0,32.0,93
888,21017,1004,557.0,56.0,94
889,22522,1065,1505.0,61.0,95


In [17]:
X = df_la.drop(labels=['confirmed_cases'], axis=1)
print (X)
y = df_la.loc[:,'confirmed_cases']
print (y)
print (y.unique())
X_col_names = X.columns.values
X_col_names

     deaths  new_confirmed_cases  new_deaths  culm_day
795       0                  0.0         0.0         1
796       0                  0.0         0.0         2
797       0                  0.0         0.0         3
798       0                  0.0         0.0         4
799       0                  0.0         0.0         5
..      ...                  ...         ...       ...
886     916                408.0        20.0        92
887     948                893.0        32.0        93
888    1004                557.0        56.0        94
889    1065               1505.0        61.0        95
890    1119                711.0        54.0        96

[96 rows x 4 columns]
795        1
796        1
797        1
798        1
799        1
       ...  
886    19567
887    20460
888    21017
889    22522
890    23233
Name: confirmed_cases, Length: 96, dtype: int64
[    1     7    11    13    14    19    20    29    32    40    54    69
    94   147   192   231   295   353   411   543   67

array(['deaths', 'new_confirmed_cases', 'new_deaths', 'culm_day'],
      dtype=object)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(X_train)
print(len(X_train),len(y_train))
print(len(X_test),len(y_test))

     deaths  new_confirmed_cases  new_deaths  culm_day
862      80                543.0        14.0        68
850       4                 58.0         2.0        56
831       0                  0.0         0.0        37
840       1                  9.0         1.0        46
848       2                 39.0         1.0        54
..      ...                  ...         ...       ...
870     244                469.0        19.0        76
804       0                  0.0         0.0        10
867     173                553.0        26.0        73
807       0                  0.0         0.0        13
832       0                  0.0         0.0        38

[67 rows x 4 columns]
67 67
29 29


In [19]:
print (X_test)
min_max_scaler = MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax 

     deaths  new_confirmed_cases  new_deaths  culm_day
835       0                  2.0         0.0        41
826       0                  0.0         0.0        32
841       1                  3.0         0.0        47
854      13                144.0         3.0        60
873     326                283.0        26.0        79
869     225                411.0        25.0        75
860      54                532.0        10.0        66
839       0                  1.0         0.0        45
875     405                442.0        39.0        81
880     619               1474.0        18.0        86
857      32                336.0         6.0        63
822       0                  0.0         0.0        28
886     916                408.0        20.0        92
872     300                308.0        33.0        78
827       0                  0.0         0.0        33
851       5                 58.0         1.0        57
834       0                  4.0         0.0        40
805       

array([[0.00000000e+00, 1.32890365e-03, 0.00000000e+00, 4.25531915e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.29787234e-01],
       [9.38967136e-04, 1.99335548e-03, 0.00000000e+00, 4.89361702e-01],
       [1.22065728e-02, 9.56810631e-02, 3.94736842e-02, 6.27659574e-01],
       [3.06103286e-01, 1.88039867e-01, 3.42105263e-01, 8.29787234e-01],
       [2.11267606e-01, 2.73089701e-01, 3.28947368e-01, 7.87234043e-01],
       [5.07042254e-02, 3.53488372e-01, 1.31578947e-01, 6.91489362e-01],
       [0.00000000e+00, 6.64451827e-04, 0.00000000e+00, 4.68085106e-01],
       [3.80281690e-01, 2.93687708e-01, 5.13157895e-01, 8.51063830e-01],
       [5.81220657e-01, 9.79401993e-01, 2.36842105e-01, 9.04255319e-01],
       [3.00469484e-02, 2.23255814e-01, 7.89473684e-02, 6.59574468e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.87234043e-01],
       [8.60093897e-01, 2.71096346e-01, 2.63157895e-01, 9.68085106e-01],
       [2.81690141e-01, 2.04651163e-01, 4.34210526e

In [21]:
df_la['culm_day']

795     1
796     2
797     3
798     4
799     5
       ..
886    92
887    93
888    94
889    95
890    96
Name: culm_day, Length: 96, dtype: int64

## RBF


In [40]:
svr_rbf = SVR(kernel='rbf', C= 100, gamma = 0.1)

In [41]:
svr_rbf.fit(X_train,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [42]:
y_predict = svr_rbf.predict(X_test)
y_predict

array([ 72.18934495,  39.91642782, 209.266168  , 256.00897476,
       256.00897169, 256.00897169, 256.00897169, 135.22981539,
       256.00897169, 256.00897169, 256.05901683,   1.46839531,
       256.00897169, 256.00897169,  53.78138709, 327.99903762,
       179.99685321,   1.14234302,   1.74971211, 239.86400619,
       257.09014555, 260.51389193,   1.13357488, 256.00897169,
       256.00897169,   8.58757791,  49.77275443, 168.64117819,
       256.00897169])

In [43]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse

8156.503967290566

# Linear

In [44]:
svr_linear = SVR(kernel='linear', C= 100, gamma = 'auto')

In [45]:
svr_linear.fit(X_train,y_train)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [46]:
y_predict = svr_linear.predict(X_test)
y_predict

array([1.06515697e+01, 1.20040917e+00, 3.18232722e+01, 8.88256435e+02,
       6.68518599e+03, 5.62338842e+03, 3.37534182e+03, 6.03226203e+00,
       8.72872411e+03, 1.70576017e+04, 2.09769138e+03, 1.15039855e+00,
       1.69596239e+04, 6.37658763e+03, 1.21291183e+00, 3.54726388e+02,
       1.99777037e+01, 9.37853389e-01, 8.37832139e-01, 2.93038377e+01,
       3.33179079e+02, 7.59327307e+02, 1.05037730e+00, 1.75996025e+04,
       2.17179408e+04, 1.25041980e+00, 1.22541449e+00, 8.32107791e+01,
       1.12723148e+04])

In [47]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse

1285.140744718964

# Polynomial


In [ ]:
svr_poly = SVR(kernel='poly', C= 100, gamma = 'auto',degree = 2)

In [ ]:
svr_poly.fit(X_train,y_train)

In [ ]:
y_predict = svr_poly.predict(X_test)
y_predict


In [ ]:
#evaluate model using mse
mse = mean_squared_error(y_test,y_predict)
#root mean square
rmse= np.sqrt(mse)
rmse